# 手書き文字のRNN(LSTM)による分類

画像の認識はCNNで行うのが主流であるが、自身の学習も兼ねて、  
手書き文字画像をあえてRNNで分類する。

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns

# tensorflow関係
import tensorflow as tf
import tflearn
import tflearn.datasets.mnist as mnist
from tflearn.datasets.mnist import DataSet

## データの取得・前処理

In [9]:
# 解凍がうまくいかない場合がある。mnist.read_data_setsをたたくとうまくいくのか
trainX, trainY, testX, testY = mnist.load_data(one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


画像データは以下のようになっている。

In [31]:
trainX[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

正解データは、数値の大小の概念がないため、one-hotエンコーディングにより以下のように表現される。  
以下は7であることがわかる。

In [30]:
trainY[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

画像ピクセルデータを1次元から系列データに変換する。

reshape関数を使ってサイズが784の一次元配列を、1次元配列28個の系列データに変換する。  
引数の-1はなんだ？

In [31]:
trainX = np.reshape(trainX, (-1, 28, 28))
testX = np.reshape(testX, (-1, 28, 28))

以下のように2次元配列に変換されたことがわかる。

In [34]:
trainX[0]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.    

## ネットワークの作成

入力層28×28ノード、中間層LSTM128ノード、出力層10ノード(0〜9 を分類するため)の  
RNNを構築する。

初期化を行う。

In [11]:
tf.reset_default_graph()

入力層を作成する。

shapeには入力する学習データの形状としてバッチサイズとノード数を設定する。  
バッチサイズは指定せず(None)、ノード数は28×28(一枚の画像のピクセル数)とする。

In [12]:
net = tflearn.input_data(shape=[None, 28, 28])

中間層(LSTMブロック)を作成する。

incoming: 結合の対象(作成する一つ前)の層  
n_units: LSTMのノード数

In [13]:
net = tflearn.lstm(incoming=net, n_units=128)

出力層(全結合層)を作成する。

incoming: 結合の対象(作成する一つ前)の層  
n_units: LSTMのノード数  
activation: 活性化関数。ここではソフトマックス関数を使用する。

In [14]:
net = tflearn.fully_connected(incoming=net, n_units=10, activation="softmax")

学習の条件を設定する。

incoming: 学習の対象となる層  
optimizer: 最適化手法。ここではsgd(確率的勾配降下法)を使用する。  
learning_rate: 学習率？  
loss: 誤差関数。ここではcategorial_clossentropy(交差エントロピー)を使用する。

In [15]:
net = tflearn.regression(incoming=net, 
                         optimizer="sgd", 
                         learning_rate=0.5, 
                         loss="categorical_crossentropy")

## モデルの学習

DNN関数により作成したRNNネットワークと学習条件をセットする。

In [16]:
model = tflearn.DNN(network=net)

fit関数を用いて学習を実行し、モデルを作成する。

X_inputs: 学習データ。ここでは学習用の画像ピクセルデータ。  
Y_targets: 正解データ。
n_epoch: 学習回数。
batch_size: バッチサイズ。　
validation_set: モデルの精度を検証するためのテストデータセットの割合。ここでは学習データセットのうち1割をテストデータとする。  
show_metric: 学習のステップごとに精度を表示するかどうかの設定。

In [17]:
model.fit(X_inputs=trainX,
          Y_targets=trainY,
          n_epoch=20,
          batch_size=1000,
          validation_set=0.1,
          show_metric=True)

---------------------------------
Run id: 0U5NNT
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 49500
Validation samples: 5500
--
Training Step: 1  | time: 1.036s
| SGD | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 01000/49500
Training Step: 2  | total loss: 2.07265 | time: 1.757s
| SGD | epoch: 001 | loss: 2.07265 - acc: 0.0927 -- iter: 02000/49500
Training Step: 3  | total loss: 2.25965 | time: 2.586s
| SGD | epoch: 001 | loss: 2.25965 - acc: 0.1036 -- iter: 03000/49500
Training Step: 4  | total loss: 2.28900 | time: 3.292s
| SGD | epoch: 001 | loss: 2.28900 - acc: 0.1324 -- iter: 04000/49500
Training Step: 5  | total loss: 2.29505 | time: 3.880s
| SGD | epoch: 001 | loss: 2.29505 - acc: 0.1390 -- iter: 05000/49500
Training Step: 6  | total loss: 2.29657 | time: 4.436s
| SGD | epoch: 001 | loss: 2.29657 - acc: 0.1474 -- iter: 06000/49500
Training Step: 

Training Step: 57  | total loss: 1.77190 | time: 3.896s
| SGD | epoch: 002 | loss: 1.77190 - acc: 0.3930 -- iter: 07000/49500
Training Step: 58  | total loss: 1.75108 | time: 4.487s
| SGD | epoch: 002 | loss: 1.75108 - acc: 0.4031 -- iter: 08000/49500
Training Step: 59  | total loss: 1.75667 | time: 5.075s
| SGD | epoch: 002 | loss: 1.75667 - acc: 0.3965 -- iter: 09000/49500
Training Step: 60  | total loss: 1.74589 | time: 5.645s
| SGD | epoch: 002 | loss: 1.74589 - acc: 0.4012 -- iter: 10000/49500
Training Step: 61  | total loss: 1.74782 | time: 6.240s
| SGD | epoch: 002 | loss: 1.74782 - acc: 0.3971 -- iter: 11000/49500
Training Step: 62  | total loss: 1.72598 | time: 6.828s
| SGD | epoch: 002 | loss: 1.72598 - acc: 0.4021 -- iter: 12000/49500
Training Step: 63  | total loss: 1.71752 | time: 7.497s
| SGD | epoch: 002 | loss: 1.71752 - acc: 0.3998 -- iter: 13000/49500
Training Step: 64  | total loss: 1.70788 | time: 8.079s
| SGD | epoch: 002 | loss: 1.70788 - acc: 0.4080 -- iter: 1400

Training Step: 114  | total loss: 1.20054 | time: 7.821s
| SGD | epoch: 003 | loss: 1.20054 - acc: 0.5826 -- iter: 14000/49500
Training Step: 115  | total loss: 1.19137 | time: 8.402s
| SGD | epoch: 003 | loss: 1.19137 - acc: 0.5849 -- iter: 15000/49500
Training Step: 116  | total loss: 1.18678 | time: 8.993s
| SGD | epoch: 003 | loss: 1.18678 - acc: 0.5862 -- iter: 16000/49500
Training Step: 117  | total loss: 1.18427 | time: 9.649s
| SGD | epoch: 003 | loss: 1.18427 - acc: 0.5832 -- iter: 17000/49500
Training Step: 118  | total loss: 1.17549 | time: 10.316s
| SGD | epoch: 003 | loss: 1.17549 - acc: 0.5862 -- iter: 18000/49500
Training Step: 119  | total loss: 1.16597 | time: 10.992s
| SGD | epoch: 003 | loss: 1.16597 - acc: 0.5878 -- iter: 19000/49500
Training Step: 120  | total loss: 1.15012 | time: 11.678s
| SGD | epoch: 003 | loss: 1.15012 - acc: 0.5964 -- iter: 20000/49500
Training Step: 121  | total loss: 1.42319 | time: 12.338s
| SGD | epoch: 003 | loss: 1.42319 - acc: 0.5465 -

Training Step: 171  | total loss: 0.77087 | time: 11.580s
| SGD | epoch: 004 | loss: 0.77087 - acc: 0.7420 -- iter: 21000/49500
Training Step: 172  | total loss: 0.75990 | time: 12.264s
| SGD | epoch: 004 | loss: 0.75990 - acc: 0.7449 -- iter: 22000/49500
Training Step: 173  | total loss: 0.75390 | time: 13.045s
| SGD | epoch: 004 | loss: 0.75390 - acc: 0.7462 -- iter: 23000/49500
Training Step: 174  | total loss: 0.74818 | time: 13.714s
| SGD | epoch: 004 | loss: 0.74818 - acc: 0.7475 -- iter: 24000/49500
Training Step: 175  | total loss: 0.73509 | time: 14.313s
| SGD | epoch: 004 | loss: 0.73509 - acc: 0.7521 -- iter: 25000/49500
Training Step: 176  | total loss: 0.72493 | time: 14.904s
| SGD | epoch: 004 | loss: 0.72493 - acc: 0.7555 -- iter: 26000/49500
Training Step: 177  | total loss: 0.71136 | time: 15.465s
| SGD | epoch: 004 | loss: 0.71136 - acc: 0.7595 -- iter: 27000/49500
Training Step: 178  | total loss: 0.70261 | time: 16.026s
| SGD | epoch: 004 | loss: 0.70261 - acc: 0.76

Training Step: 228  | total loss: 0.86053 | time: 15.908s
| SGD | epoch: 005 | loss: 0.86053 - acc: 0.7723 -- iter: 28000/49500
Training Step: 229  | total loss: 0.81600 | time: 16.472s
| SGD | epoch: 005 | loss: 0.81600 - acc: 0.7836 -- iter: 29000/49500
Training Step: 230  | total loss: 0.78014 | time: 17.042s
| SGD | epoch: 005 | loss: 0.78014 - acc: 0.7918 -- iter: 30000/49500
Training Step: 231  | total loss: 0.74078 | time: 17.600s
| SGD | epoch: 005 | loss: 0.74078 - acc: 0.8015 -- iter: 31000/49500
Training Step: 232  | total loss: 0.70526 | time: 18.166s
| SGD | epoch: 005 | loss: 0.70526 - acc: 0.8102 -- iter: 32000/49500
Training Step: 233  | total loss: 0.67502 | time: 18.758s
| SGD | epoch: 005 | loss: 0.67502 - acc: 0.8177 -- iter: 33000/49500
Training Step: 234  | total loss: 0.64883 | time: 19.308s
| SGD | epoch: 005 | loss: 0.64883 - acc: 0.8236 -- iter: 34000/49500
Training Step: 235  | total loss: 0.62384 | time: 19.918s
| SGD | epoch: 005 | loss: 0.62384 - acc: 0.82

...

Training Step: 741  | total loss: 0.43003 | time: 23.673s
| SGD | epoch: 015 | loss: 0.43003 - acc: 0.9276 -- iter: 41000/49500
Training Step: 742  | total loss: 0.40008 | time: 24.321s
| SGD | epoch: 015 | loss: 0.40008 - acc: 0.9315 -- iter: 42000/49500
Training Step: 743  | total loss: 0.37094 | time: 24.956s
| SGD | epoch: 015 | loss: 0.37094 - acc: 0.9360 -- iter: 43000/49500
Training Step: 744  | total loss: 0.34481 | time: 25.627s
| SGD | epoch: 015 | loss: 0.34481 - acc: 0.9398 -- iter: 44000/49500
Training Step: 745  | total loss: 0.31950 | time: 26.307s
| SGD | epoch: 015 | loss: 0.31950 - acc: 0.9439 -- iter: 45000/49500
Training Step: 746  | total loss: 0.29874 | time: 26.926s
| SGD | epoch: 015 | loss: 0.29874 - acc: 0.9468 -- iter: 46000/49500
Training Step: 747  | total loss: 0.27913 | time: 27.507s
| SGD | epoch: 015 | loss: 0.27913 - acc: 0.9497 -- iter: 47000/49500
Training Step: 748  | total loss: 0.26137 | time: 28.071s
| SGD | epoch: 015 | loss: 0.26137 - acc: 0.95

Training Step: 798  | total loss: 0.09431 | time: 28.000s
| SGD | epoch: 016 | loss: 0.09431 - acc: 0.9738 -- iter: 48000/49500
Training Step: 799  | total loss: 0.09398 | time: 28.590s
| SGD | epoch: 016 | loss: 0.09398 - acc: 0.9740 -- iter: 49000/49500
Training Step: 800  | total loss: 0.09592 | time: 30.248s
| SGD | epoch: 016 | loss: 0.09592 - acc: 0.9734 | val_loss: 0.10003 - val_acc: 0.9731 -- iter: 49500/49500
--
Training Step: 801  | total loss: 0.09367 | time: 0.557s
| SGD | epoch: 017 | loss: 0.09367 - acc: 0.9743 -- iter: 01000/49500
Training Step: 802  | total loss: 0.09162 | time: 1.124s
| SGD | epoch: 017 | loss: 0.09162 - acc: 0.9746 -- iter: 02000/49500
Training Step: 803  | total loss: 0.08972 | time: 1.701s
| SGD | epoch: 017 | loss: 0.08972 - acc: 0.9754 -- iter: 03000/49500
Training Step: 804  | total loss: 0.08875 | time: 2.274s
| SGD | epoch: 017 | loss: 0.08875 - acc: 0.9761 -- iter: 04000/49500
Training Step: 805  | total loss: 0.08824 | time: 2.846s
| SGD | ep

Training Step: 855  | total loss: 0.20951 | time: 2.901s
| SGD | epoch: 018 | loss: 0.20951 - acc: 0.9559 -- iter: 05000/49500
Training Step: 856  | total loss: 0.19646 | time: 3.469s
| SGD | epoch: 018 | loss: 0.19646 - acc: 0.9583 -- iter: 06000/49500
Training Step: 857  | total loss: 0.18452 | time: 4.032s
| SGD | epoch: 018 | loss: 0.18452 - acc: 0.9602 -- iter: 07000/49500
Training Step: 858  | total loss: 0.17573 | time: 4.605s
| SGD | epoch: 018 | loss: 0.17573 - acc: 0.9618 -- iter: 08000/49500
Training Step: 859  | total loss: 0.16706 | time: 5.169s
| SGD | epoch: 018 | loss: 0.16706 - acc: 0.9629 -- iter: 09000/49500
Training Step: 860  | total loss: 0.15938 | time: 5.786s
| SGD | epoch: 018 | loss: 0.15938 - acc: 0.9644 -- iter: 10000/49500
Training Step: 861  | total loss: 0.15016 | time: 6.359s
| SGD | epoch: 018 | loss: 0.15016 - acc: 0.9666 -- iter: 11000/49500
Training Step: 862  | total loss: 0.14503 | time: 6.927s
| SGD | epoch: 018 | loss: 0.14503 - acc: 0.9672 -- it

Training Step: 912  | total loss: 0.07844 | time: 7.086s
| SGD | epoch: 019 | loss: 0.07844 - acc: 0.9785 -- iter: 12000/49500
Training Step: 913  | total loss: 0.07543 | time: 7.647s
| SGD | epoch: 019 | loss: 0.07543 - acc: 0.9793 -- iter: 13000/49500
Training Step: 914  | total loss: 0.07490 | time: 8.212s
| SGD | epoch: 019 | loss: 0.07490 - acc: 0.9791 -- iter: 14000/49500
Training Step: 915  | total loss: 0.07425 | time: 8.797s
| SGD | epoch: 019 | loss: 0.07425 - acc: 0.9799 -- iter: 15000/49500
Training Step: 916  | total loss: 0.07582 | time: 9.365s
| SGD | epoch: 019 | loss: 0.07582 - acc: 0.9793 -- iter: 16000/49500
Training Step: 917  | total loss: 0.07726 | time: 9.686s
| SGD | epoch: 019 | loss: 0.07726 - acc: 0.9791 -- iter: 17000/49500
Training Step: 918  | total loss: 0.07527 | time: 10.005s
| SGD | epoch: 019 | loss: 0.07527 - acc: 0.9798 -- iter: 18000/49500
Training Step: 919  | total loss: 0.07151 | time: 10.615s
| SGD | epoch: 019 | loss: 0.07151 - acc: 0.9812 -- 

Training Step: 969  | total loss: 0.11406 | time: 10.825s
| SGD | epoch: 020 | loss: 0.11406 - acc: 0.9733 -- iter: 19000/49500
Training Step: 970  | total loss: 0.10725 | time: 11.397s
| SGD | epoch: 020 | loss: 0.10725 - acc: 0.9749 -- iter: 20000/49500
Training Step: 971  | total loss: 0.10426 | time: 11.962s
| SGD | epoch: 020 | loss: 0.10426 - acc: 0.9754 -- iter: 21000/49500
Training Step: 972  | total loss: 0.10268 | time: 12.536s
| SGD | epoch: 020 | loss: 0.10268 - acc: 0.9757 -- iter: 22000/49500
Training Step: 973  | total loss: 0.09888 | time: 13.099s
| SGD | epoch: 020 | loss: 0.09888 - acc: 0.9766 -- iter: 23000/49500
Training Step: 974  | total loss: 0.09684 | time: 13.669s
| SGD | epoch: 020 | loss: 0.09684 - acc: 0.9766 -- iter: 24000/49500
Training Step: 975  | total loss: 0.09505 | time: 14.245s
| SGD | epoch: 020 | loss: 0.09505 - acc: 0.9767 -- iter: 25000/49500
Training Step: 976  | total loss: 0.09502 | time: 14.811s
| SGD | epoch: 020 | loss: 0.09502 - acc: 0.97

## 予測

学習させたモデルを用いてテスト用画像ピクセルデータの予測を行う。

それぞれの出力結果は0〜9とする。

In [35]:
pred = np.array(model.predict(testX)).argmax(axis=1)
pred

array([7, 2, 1, ..., 4, 5, 6])

テスト用データ正解を値をlabelに格納する。

In [36]:
label = testY.argmax(axis=1)
label

array([7, 2, 1, ..., 4, 5, 6])

In [37]:
accuracy = np.mean(pred == label, axis=0)
accuracy

0.97709999999999997

## 学習済みモデルの保存と読み込み

学習済みのモデルの保存や読み込みを行うことができる。  
読み込んだモデルは同様にテストデータに対して予測を行うことができる。

In [30]:
model.save("handwritten_rnn")

INFO:tensorflow:/Users/fukunaritakeshi/PycharmProjects/python-ml-programming/handwritten_rnn is not in all_model_checkpoint_paths. Manually adding it.


In [26]:
new_model = model.load("handwritten_rnn")

INFO:tensorflow:Restoring parameters from /Users/fukunaritakeshi/PycharmProjects/python-ml-programming/handwritten_rnn


In [27]:
new_pred = np.array(model.predict(testX)).argmax(axis=1)
label = testY.argmax(axis=1)
new_accuracy = np.mean(new_pred == label, axis=0)
new_accuracy

0.9771
